# С1. Модуль 1 Загрузка, предобработка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
import sklearn
import catboost
import statsmodels
import scipy.stats
from joblib import Parallel, delayed
import soundfile as sf
from glob import glob
from tqdm import tqdm

Для начала работы необходимо найти пути к файлам определенного формата. Ниже представлен код, позволяющий определить пути к файлам.

In [ ]:
filenames_dict = {}
for format_ in ['csv', 'xlsx', 'wav', 'mkv']:
    filenames = glob('./data/**.' + format_, recursive=True)
    filenames_dict[format_] = filenames
filenames_dict

{'csv': ['./data\\13.csv',
  './data\\14.csv',
  './data\\17.csv',
  './data\\24.csv',
  './data\\26.csv',
  './data\\36.csv',
  './data\\38.csv',
  './data\\46.csv',
  './data\\50.csv',
  './data\\52.csv',
  './data\\55.csv',
  './data\\56.csv',
  './data\\59.csv',
  './data\\68.csv',
  './data\\7.csv',
  './data\\71.csv',
  './data\\8.csv',
  './data\\9.csv'],
 'xlsx': ['./data\\1.xlsx',
  './data\\10.xlsx',
  './data\\12.xlsx',
  './data\\15.xlsx',
  './data\\16.xlsx',
  './data\\19.xlsx',
  './data\\2.xlsx',
  './data\\20.xlsx',
  './data\\21.xlsx',
  './data\\22.xlsx',
  './data\\23.xlsx',
  './data\\25.xlsx',
  './data\\27.xlsx',
  './data\\3.xlsx',
  './data\\31.xlsx',
  './data\\32.xlsx',
  './data\\33.xlsx',
  './data\\34.xlsx',
  './data\\35.xlsx',
  './data\\37.xlsx',
  './data\\39.xlsx',
  './data\\4.xlsx',
  './data\\41.xlsx',
  './data\\42.xlsx',
  './data\\43.xlsx',
  './data\\44.xlsx',
  './data\\45.xlsx',
  './data\\47.xlsx',
  './data\\48.xlsx',
  './data\\49.xlsx',
 

Таким образом, имеем словарь, где ключ - формат файла, значение - пути к файлам для дальнейшей загрузки.

### Считывание csv файлов

Поскольку размер csv файлов большой, необходимо перед объединением в одну таблицу переконвертировать тип данных в формат, требующий меньше памяти для хранения (в нашем случае, float32). Однако, поскольку оперативной памяти компьютера недостаточно для хранения такого большого объема файлов, а для распределенной загрузки и хранения необходимо большее время, файлы были загружены частично.

При очередной попытке загрузки файлов машина сломалась, пришлось перезагружать компьютер :(

При предварительном анализе было выявлено, что файлы в первых четырех строках содержат техническую информацию, которую при загрузке было решено удалить

Ниже представлена функция для загрузки файлов с вышесказанными опциями

In [ ]:
def read_csv_file(path):
    df = pd.read_csv(path, skiprows=[0, 1, 2, 3])
    if len(df.columns) != 1:
        df.drop(columns=df.columns[1], inplace=True)
    df.columns = ['csv_values']
    df['path'] = len(df) * [int(path.split('\\')[-1].split('.')[0])]
    df['path'] = df['path'].astype('int32')
    df['csv_values'] = df['csv_values'].astype('float32')
    return df

In [ ]:
data_csv = pd.DataFrame()
for path in tqdm(filenames_dict['csv']):
    try:
        df = read_csv_file(path)
        data_csv = pd.concat([data_csv, df])
        data_csv.to_csv('./data_saved/data_csv.csv', index=False)
    except:
        print('Data is not loaded')
        continue

 28%|██████████████████████▊                                                           | 5/18 [09:41<34:55, 161.20s/it]

Data is not loaded


 33%|███████████████████████████▎                                                      | 6/18 [09:45<21:30, 107.56s/it]

Data is not loaded


 39%|████████████████████████████████▎                                                  | 7/18 [09:46<13:21, 72.86s/it]

...

Data is not loaded


 94%|█████████████████████████████████████████████████████████████████████████████▍    | 17/18 [09:52<00:02,  2.44s/it]

Data is not loaded


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [09:54<00:00, 33.05s/it]

Data is not loaded


Поскольку ноутбук пришлось перезагружать, данные по csv файлам не сохранились

In [ ]:
data_csv.to_csv('./data_saved/data_csv.csv', index=False)

KeyboardInterrupt: 

Однако удалось сделать быстрый анализ и выявить аномальные значения. Они соответствуют таким значениям, как "99999", "!@&%*#(+
" и так далее

In [ ]:
data_csv

,csv_values,path
0,1.518057,13
1,1.573271,13
2,1.617384,13
3,1.643969,13
4,1.666464,13
...,...,...
1421277,-219.775879,26
1421278,-222.539566,26
1421279,-224.994675,26
1421280,-226.048706,26


При анализе данных было выявлены аномалии в виде нечитаемых значений

### Считывание эксель файлов

Эксель формат при считывании автоматически ставит в заголовок первое значение, поэтому в функции аргумент header задан как None. Для привзяки к остальным данным также добавлена информация о пути файла.

In [ ]:
data_excel = pd.DataFrame()
for path in tqdm(filenames_dict['xlsx']):
    df = pd.read_excel(path, header=None)
    df.columns = ['excel_values']
    df['path'] = len(df) * [path]
    data_excel = pd.concat([data_excel, df])

Сохранение таблицы для дальнейшего доступа

In [ ]:
data_excel.to_csv('./data_saved/data_excel.csv', index=False)

### Считывание аудио

Для считывания аудиозаписи использовалась библиотека soundfile, возвращающая количество дискретов и список, представляющий сигнал аудиозаписи. Для привзяки к остальным данным также добавлена информация о пути файла. Данные сохранены.

In [ ]:
data_audio = np.array([])
for path in tqdm(filenames_dict['wav']):
    audio = sf.read(path)
    audio_info = np.array([audio, path])
    data_audio = np.append(data_audio, sf.read(path))

In [ ]:
with open('./data_saved/data_audio.npy', 'wb') as f:
    np.save(f, data_audio)

### Считывание видео

При анализе данных по видео было выявлено, что частота кадров не соответствует заявленной (60 в секунду). Длина первого видео составляет 4 минуты 42 секунды, 282 секунды. Количество кадров в этмо видео - 8460. 8460 / 282 = 30. Таким образом, **частота кадров видео составляет 30 секунд**

Поскольку не было найдено достоверной информации, как получить вектор температуры из видео, было решено применить аналитический подход. Каждый кадр видео представляет из себя массив из трех каналов. Для каждого кадра по каждому каналу было взято среднее арифметическое. Таким образом, данные по "векторам температуры" представляют из себя массив из усредененных значенией по каналу: m * n * z, где m - значение температуры, n - номер канала (0 - красный, 1 - зеленый, 2 - синий), z - секунды 

In [ ]:
data_video = []
for path in tqdm(filenames_dict['mkv']):
    video = cv.VideoCapture(path)
    single_video = []
    success, image = video.read()
    count = 0
    while success:
        success, image = video.read()
        if count % 30 == 0:
            image = image[:570, :780]
            values = [np.mean(image[:, :, k]) for k in range(3)]
            single_video.append(values)
            count += 1
        else:
            count += 1
            continue
    data_video.append([path, np.array(single_video)])

In [ ]:
with open('./data_saved/data_video.npy', 'wb') as f:
    np.save(f, data_video)